## Project 5 - Data Cleaning

### Import libraries

In [1]:
import pandas as pd
import math

pd.set_option('display.max_rows', 10)

### Load data

In [2]:
df = pd.read_csv('../data/raw_yemen_data.csv', encoding='cp1252')

### Checkout null values

In [3]:
df.isnull().sum()

Incident ID        0
Date               0
Governorate        0
District         238
Area            3299
               ...  
Time of Day        0
Unnamed: 19    22360
Unnamed: 20    22360
Unnamed: 21    22360
Unnamed: 22    22436
Length: 23, dtype: int64

**Drop Columns:** 
- Unnamed: 19 
- Unnamed: 20 
- Unnamed: 21 
- Unnamed: 22 

These columns are mainly missing values and the information in the non-missing cells is duplicated in other columns.

In [4]:
df.drop(columns = ['Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'], inplace=True)

In [5]:
len(df['Area'].unique())

5805

In [6]:
#df = df.drop(columns=['Unnamed: 19', 'Unnamed: 20',
#       'Unnamed: 21', 'Unnamed: 22'])

In [7]:
df['Civilian Casualties'].value_counts()

0      20667
2        224
5        212
1        199
3        194
       ...  
77         1
101        1
85         1
100        1
95         1
Name: Civilian Casualties, Length: 89, dtype: int64

In [8]:
df['Main category'].value_counts()

Unknown                     8496
Military_Security_Target    7385
Civilian                    2879
Infrastructure              1447
Economic_infrastructure     1080
                            ... 
international_community       13
Political_Tribal              11
International_community        6
cultural_heritage              2
International_Community        1
Name: Main category, Length: 18, dtype: int64

### Clean up columns

#### Column names

In [9]:
# Clean up column names: Remove spaces, make lowercase, replace space with underscore
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-','_')

In [10]:
df.columns

Index(['incident_id', 'date', 'governorate', 'district', 'area', 'target',
       'main_category', 'sub_category', 'min_air_raids', 'max_air_raids',
       'civilian_casualties', 'fatalities', 'woman_fatalities',
       'child_fatalities', 'injured', 'woman_injured', 'child_injured',
       'confirmed_time', 'time_of_day'],
      dtype='object')

#### Column values


date: Make index and convert to datetime

In [11]:

df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index


DatetimeIndex(['2015-03-26', '2015-03-26', '2015-03-26', '2015-03-26',
               '2015-03-26', '2015-03-26', '2015-03-26', '2015-03-26',
               '2015-03-26', '2015-03-26',
               ...
               '2020-12-29', '2020-12-30', '2020-12-30', '2020-12-30',
               '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31',
               '2020-12-31', '2020-12-31'],
              dtype='datetime64[ns]', name='date', length=22485, freq=None)

incident id: Check for duplicates

In [12]:
any([x > 1 for x in df['incident_id'].value_counts()])

False

governorate: Check value counts.

In [13]:
df['governorate'].value_counts()

Saada        5199
Taiz         2661
Sanaa        2568
Hajja        2429
Marib        2248
             ... 
Raymah         12
Hadramawt       8
Maharah         2
lahj            2
Marib           2
Name: governorate, Length: 23, dtype: int64

Check 'Capital' - There is 'Capital' which may be 'Sanaa' or 'Aden'
(The capital is currently disputed in Yemen, historically it is Sanaa but recently it's Aden)

Resesarch on district in relation to capital
* if distric is bani al-harith, then capital = Sanaa
* if district is al-sab'ein, then capital = Sanaa
* if ma'ain then Sanaa
* if al-wahdah then Sanaa
* if azal then Sanaa
* shu'oub near Sanaa

In [14]:
df[df['governorate']== 'Capital']

,incident_id,governorate,district,area,target,main_category,sub_category,min_air_raids,max_air_raids,civilian_casualties,fatalities,woman_fatalities,child_fatalities,injured,woman_injured,child_injured,confirmed_time,time_of_day
date,,,,,,,,,,,,,,,,,,
2015-03-26,1,Capital,Bani Al-Harith,Al-Rahabah,Al-Daylami Airbase,Military_Security_Target,Military Site,1,2,0,0,0,0,0,0,0.0,2,Early-Morning
2015-03-26,2,Capital,Bani Al-Harith,Airport,Sana'a International Airport,Infrastructure,Transport,1,2,0,0,0,0,0,0,0.0,2,Early-Morning
2015-03-26,3,Capital,Bani Al-Harith,Al-Sonblah Neighbourhood,Residential Area,Civilian,Residential Area,1,2,29,21,3,14,8,0,7.0,2,Early-Morning
2015-03-26,4,Capital,Al-Sab'ein,Al-Nahdain,Presidential Palace,Military_Security_Target,Military Site,1,2,0,0,0,0,0,0,0.0,3,Early-Morning
2015-03-26,5,Capital,Al-Thawrah,Al-Nahdhah,Former 1st Armoured Division,Military_Security_Target,Military Site,1,2,0,0,0,0,0,0,0.0,3,Early-Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-12,22368,Capital,Bani Al-Harith,Ar Rahabah,Sana'a International Airport,Infrastructure,Transport,1,2,0,0,0,0,0,0,0.0,12,Midday
2020-04-12,22377,Capital,Bani Al-Harith,Ar Rahabah,Sana'a International Airport,Infrastructure,Transport,1,1,0,0,0,0,0,0,0.0,16,Afternoon
2020-12-29,22471,Capital,Bani Al-Harith,Ar Rahabah,Sana'a International Airport,Infrastructure,Transport,1,2,0,0,0,0,0,0,0.0,13,Midday


In [15]:
# Rename Capital cells to Sanaa
df['governorate'] = df['governorate'].replace(['Capital'], 'Sanaa')

district: clean strings

In [16]:
district_list = []
for value in df['district']:
    value = str(value).lower().strip()
    district_list.append(value)

df['district'] = district_list

area: Change null values to string 'Unknown'.

In [17]:
df['area'] = df['area'].fillna('Unknown')

target: Show values of Target

In [18]:
# target
df['target'].value_counts()


Unknown                                      8364
Pro-Houthi forces                            2123
Residential Area                              590
Pro-Houthi sites                              403
Pro-Houthi positions                          396
                                             ... 
insurance building                              1
14th Armoured Brigade                           1
House ( belongs to Ali Saleh Al-Ahmar )         1
House  (belongs to Badruldean Al-Houthi )       1
Port road                                       1
Name: target, Length: 2884, dtype: int64

main_catagory:  Clean values to better display in graphs and further eda.

In [19]:
# main_category
main_category_list = []
for value in df['main_category']:
    value = value.lower().strip()
    main_category_list.append(value)
df['main_category'] = main_category_list


sub_catagory:  Clean values to better display in graphs and further eda.

In [20]:
# sub-category
sub_category_list = []
for value in df['sub_category']:
    value = value.lower().strip()
    sub_category_list.append(value)
df['sub_category'] = sub_category_list

min air raids:  show null values

In [21]:
# min_air_raids
df['min_air_raids'].isnull().sum()

0

max_air_raids: Clean values. We understood the max air raids value to be a baseline estimate by the Yemen Data Project.
The noted that some air raids mave have had more than the number listed and marked those values with a (+) sign.  We
removed these signs for our data and used their baseline known value for max air raids as our value. We also know that
there are 2 null values in the data.  We are replacing those two values with the median number of airraids as to not
lose data.

In [22]:
# max_air_raids
max_air_raids = []
for value in df['max_air_raids']:
    if value == '2,2':  # This was an error in the csv file
        val = 2
    elif value != '-' and type(value) != float:
        val = int(value.strip('+'))
    elif type(value) == float:
        val = value
    else:
        val = 0
    max_air_raids.append(val)
df['max_air_raids'] = max_air_raids
df['max_air_raids'] = df['max_air_raids'].fillna(df['max_air_raids'].median())



civilian_casualties:  show null values

In [23]:
# civilian_casualties
df['civilian_casualties'].isnull().sum()

0

fatalities:  show null values

In [24]:
# fatalities
df['fatalities'].isnull().sum()

0

woman_fatalities:  show null values

In [25]:
# woman_fatalities
df['woman_fatalities'].isnull().sum()


0

child_fatalities:  show null values

In [26]:
# child_fatalities
df['child_fatalities'].isnull().sum()

0

injured:  show null values

In [27]:
# injured
df['injured'].isnull().sum()

0

woman_injured:  show null values

In [28]:
# woman_injured
df['woman_injured'].isnull().sum()

0

child_injured:  There are 2 null values for child injured. We are replacing those values with the median.

In [29]:
# child_injured
df['child_injured'] = df['child_injured'].fillna(df['child_injured'].median())

confirmed_time: Change multiple formats for 'Unknown'. To one uniform notation.

In [30]:
# confirmed_time
df['confirmed_time'] = df['confirmed_time'].replace(
    {'Unkn': 'Unknown', 'unkn': 'Unknown', '41': 'Unknown'}).fillna('18')

In [31]:
df[df['confirmed_time'] == 'Unknown']

,incident_id,governorate,district,area,target,main_category,sub_category,min_air_raids,max_air_raids,civilian_casualties,fatalities,woman_fatalities,child_fatalities,injured,woman_injured,child_injured,confirmed_time,time_of_day
date,,,,,,,,,,,,,,,,,,
2015-03-26,9,Sanaa,arhab,Al-Sama'a,Al-Sama'a Military Camp,military_security_target,military site,1,2.0,0,0,0,0,0,0,0.0,Unknown,Afternoon
2015-03-26,19,Hajja,washhah,-,Haqqah School,educational_facility,school,1,2.0,0,0,0,0,0,0,0.0,Unknown,Afternoon
2015-03-26,20,Hajja,washhah,"Samir, Shammakh mountains",Unknown,unknown,unknown,1,2.0,0,0,0,0,0,0,0.0,Unknown,Afternoon
2015-03-26,23,Saada,haydan,Marran,Houses (reportedly belonging to the Houthis le...,political_tribal,figure,1,2.0,0,0,0,0,0,0,0.0,Unknown,Early-Morning
2015-03-26,24,Saada,haydan,Maa,Weapons Storage,military_security_target,weapon storage,1,2.0,0,0,0,0,0,0,0.0,Unknown,Early-Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,22450,Bayda,al-maljim,Unknown,Pro-Houthi reinforcements,military_security_target,moving target (weapons/fighters),1,2.0,0,0,0,0,0,0,0.0,Unknown,Unknown
2020-12-24,22451,Hajja,hayran,Unknown,Pro-Houthi positions,military_security_target,forces,1,2.0,0,0,0,0,0,0,0.0,Unknown,Unknown
2020-12-25,22453,Marib,jabal murad,Unknown,Pro-Houthi positions,military_security_target,forces,1,3.0,0,0,0,0,0,0,0.0,Unknown,Unknown


In [32]:
#confirmed_time_list = []
#for value in list(zip(df['confirmed_time'], df['time_of_day'])):
#    if value[0] != 'Unknown' or type(value[0]) == object or type(value[0]) == str:
#        val = value[0]
#    elif math.isnan(value[0]) == True:
#        val = 18
#    elif value[0] == 'Unknown':
#        if value[1].lower().strip() == 'early-morning':
#            val = 3
#        elif value[1].lower().strip() == 'morning':
#            val = 7
#        elif value[1].lower().strip() == 'midday':
#            val = 11 
#        elif value[1].lower().strip() == 'afternoon':
#            val = 14
#        elif value[1].lower().strip() == 'evening':
#            val = 18
#        elif value[1].lower().strip() == 'night':
#            val = 22
#    confirmed_time_list.append(val)
#
#df['confirmed_time'] = confirmed_time_list

#Anyone wants to mess with this to fill 146/22500 values feel free haha

time_of_day: We cleaned the data, then replaced more specific timeframes to a simpler (morning, afternoon, night) format.

In [33]:
# time_of_day 
df['time_of_day'] = df['time_of_day'].str.lower()
df['time_of_day'] = df['time_of_day'].replace([' midday'], 'afternoon')
df['time_of_day'] = df['time_of_day'].replace(['midday'], 'afternoon')
df['time_of_day'] = df['time_of_day'].replace(['evening'], 'night')
df['time_of_day'] = df['time_of_day'].replace(['early-morning'], 'morning')
df['time_of_day'] = df['time_of_day'].replace(['early morning'], 'morning')

In [35]:
# Save cleaned dataframe to csv in data directory for use in modeling.
df.to_csv('../data/clean_df.csv')